In [ ]:
!ls -a

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.io_model as io_m

In [ ]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path

In [ ]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

In [ ]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/'

In [ ]:
print(GOOGLE_DRIVE_PATH)

In [ ]:
df = rd.read_horse_csv(DATA_PATH)

In [ ]:
df.info()

すぐに使えそうな特徴量
* frame_number, burden_weight, odds, popular

加工が必要な特徴量
* horse_id, sex_and_age, rider_id, half_way_rank, horse_weight, tamer_id

## 前処理

### ラベルの作成
問題は２値分類 (ラベル: 1 => 1~3着, 0 => 4着以降)

In [ ]:
def make_label(rank):
    return [1 if r in ["1", "2", "3"] else 0 for r in rank]

In [ ]:
df["label"] = make_label(df["rank"].values)

In [ ]:
df.head()

In [ ]:
# 欠損値？
df = df.replace('---', 0)
df = df.sample(frac=1, random_state=10)

In [ ]:
# 欠損値の確認
df.isnull().sum()

In [ ]:
df["label"].value_counts()

In [ ]:
# 学習に用いるデータセットの作成
x = np.array(df[["frame_number", "burden_weight"]]).astype(np.float32)
y = np.array(df["label"]).astype(np.float32)
del df
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=11)
del x, y

In [ ]:
# データセットのシャッフルとバッチ化
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(1024)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1024)

In [ ]:
import tensorflow as tf
import utils.sample_model as model
    
# モデルのインスタンスを作成
model = model.HorseModel(x_train.shape[1])

In [ ]:
metrics = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=metrics)

## 評価指標のメモ
(陽性: 1~3着, 陰性: 4着~最下位)

* tp: 真陽性の数
* fp: 偽陽性の数
* tn: 真陰性の数
* fn: 偽陰性の数
以下の指標は1に近いほど、モデルが良い性能を持っていることを示す。
* accuracy: 正解率
* precision: 適合率 <br>
  3着以内と予測してどれだけ当たっているか？(陽性だった予測値の正解率 $\frac{tp}{tp + fp}$)<br>
* recall: 再現率 <br>
  3着以内のデータをどれだけ当てられているか?(正解が陽性であるデータの正解率 $\frac{tp}{tp + np}$)
* auc: AUC<br>
  0.5に近いと予測がランダムであることを示している

In [ ]:
model.fit(train_ds, epochs=100) 

In [ ]:
model.evaluate(test_ds, verbose=2)

In [ ]:
# テストデータの予測値と正解ラベルの確認
for pred, y in zip(model.predict(x_test), y_test):
    print(f"pred: {pred}, label: {y}")

In [ ]:
# モデルの保存
io_m.save_model(model, model_name="first_model")

In [ ]:
# 保存したモデルに不具合がないか確認
model = io_m.read_model("first_model")
model.evaluate(test_ds, verbose=2)